In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sys
import json
from datetime import datetime
from tqdm import tqdm
import pytz
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# training and Testing directories
training_dir = os.path.join("Datasets", "Training")
testing_dir = os.path.join("Datasets", "Testing")
if not os.path.isdir(training_dir):
    raise Exception("ERROR: training dataset not found")
if not os.path.isdir(testing_dir):
    raise Exception("ERROR: testing dataset not found")

In [3]:
# iterate over all hashtag files 
for root, dirs, files in os.walk(training_dir, topdown=False):
    for file in files:
        print(os.path.join(root, file))

Datasets/Training/tweets_#gohawks.txt
Datasets/Training/tweets_#gopatriots.txt
Datasets/Training/tweets_#nfl.txt
Datasets/Training/tweets_#patriots.txt
Datasets/Training/tweets_#sb49.txt
Datasets/Training/tweets_#superbowl.txt


In [4]:
# Initialize dictionaries and Unix times for Feb 1, 8 am and Feb 1, 8 pm. 
# Dictionary keys: hashtag.
# Dictionary values: [time of tweet (Unix), number of retweets for tweet, number of followers for tweeter]

hashtag_dict_before = {}
hashtag_dict_during = {}
hashtag_dict_after = {}
start_unix_time = 1422806400
end_unix_time = 1422849600
pst_tz = pytz.timezone('America/Los_Angeles')


In [5]:
for root, dirs, files in os.walk(training_dir, topdown=False):
    for file in files:
        filename = os.path.splitext(file)[0].replace('tweets_#', '')
        print('Parsing {}...'.format(filename))
        
        hashtag_dict_before[filename] = []
        hashtag_dict_during[filename] = []
        hashtag_dict_after[filename] = []
        
        # open the file and read all lines:
        with open(os.path.join(root, file), "r", encoding="utf-8") as hashtag:
            # read line-by-line
            for line in hashtag:
                json_obj = json.loads(line)
                
                # get desired statistics
                citation_date = json_obj['citation_date'] # Unix time
                num_retweets = json_obj['metrics']['citations']['total'] # Number of retweets for this tweet
                num_followers = json_obj['author']['followers'] # Number of followers for tweeter
                
                if citation_date < start_unix_time:
                    hashtag_dict_before[filename].append([citation_date, num_retweets, num_followers])
                elif citation_date > end_unix_time:
                    hashtag_dict_after[filename].append([citation_date, num_retweets, num_followers])
                else:
                    hashtag_dict_during[filename].append([citation_date, num_retweets, num_followers])
    print('done')

Parsing gohawks...
Parsing gopatriots...
Parsing nfl...
Parsing patriots...
Parsing sb49...
Parsing superbowl...
done


In [6]:
# Sort each by time.

hashtags = ['gohawks', 'gopatriots', 'nfl', 'patriots', 'sb49', 'superbowl']

for key in hashtags:
    hashtag_dict_before[key] = np.array(hashtag_dict_before[key])
    indices = np.argsort(hashtag_dict_before[key][:,0])
    hashtag_dict_before[key] = hashtag_dict_before[key][indices]
    
    hashtag_dict_during[key] = np.array(hashtag_dict_during[key])
    indices = np.argsort(hashtag_dict_during[key][:,0])
    hashtag_dict_during[key] = hashtag_dict_during[key][indices]

    hashtag_dict_after[key] = np.array(hashtag_dict_after[key])
    indices = np.argsort(hashtag_dict_after[key][:,0])
    hashtag_dict_after[key] = hashtag_dict_after[key][indices]


In [7]:
# Find how many time windows there are

num_windows_before = int(np.max([((start_unix_time - hashtag_dict_before[key][0,0]) // 3600) + 1 for key in hashtags]))
num_windows_during = int(np.max([((end_unix_time - start_unix_time) // 3600 * 12) + 1 for key in hashtags]))
num_windows_after = int(np.max([((hashtag_dict_after[key][-1,0] - end_unix_time) // 3600) + 1 for key in hashtags]))

In [85]:
data_hashtag_before = {}
data_hashtag_during = {}
data_hashtag_after = {}

for key in hashtags:
    print(key)
    
    # Rename the dictionary value for readability
    temp_before = hashtag_dict_before[key]
    temp_during = hashtag_dict_during[key]
    temp_after = hashtag_dict_after[key]
        
    # Iterate through all elements before start time
    data_hashtag_before[key] = np.zeros((num_windows_before, 5))
    num_followers_before = {}
    for i in range(np.shape(temp_before)[0]):
        item_before = int((start_unix_time - temp_before[i,0]) // 3600)
        data_hashtag_before[key][item_before] += np.array([1, int(temp_before[i, 1]), int(temp_before[i, 2]), 0, 0])
#         data_hashtag_before[key][item_before][3] = np.max(temp_before[:,2])
        dt_obj_pst = datetime.fromtimestamp(temp_before[i,0], pst_tz)
        data_hashtag_before[key][item_before][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_before not in num_followers_before.keys():
            num_followers_before[item_before] = []
        num_followers_before[item_before].append(temp_before[i,2])
    for i in num_followers_before.keys():
        data_hashtag_before[key][i][3] = np.max(num_followers_before[i])
        
    # Iterate through all elements during time
    data_hashtag_during[key] = np.zeros((num_windows_during, 5))
    num_followers_during = {}
    for i in range(np.shape(temp_during)[0]):
        item_during = int(((end_unix_time - temp_during[i,0]) * 12) // 3600)
        data_hashtag_during[key][item_during] += np.array([1, int(temp_during[i, 1]), int(temp_during[i, 2]), 0, 0])
#         data_hashtag_during[key][item_during][3] = np.max(temp_during[:,2])
        dt_obj_pst = datetime.fromtimestamp(temp_during[i,0], pst_tz)
        data_hashtag_during[key][item_during][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_during not in num_followers_during.keys():
            num_followers_during[item_during] = []
        num_followers_during[item_during].append(temp_during[i,2])
    for i in num_followers_during.keys():
        data_hashtag_during[key][i][3] = np.max(num_followers_during[i])
        
    # Iterate through all elements after end time
    data_hashtag_after[key] = np.zeros((num_windows_after, 5))
    num_followers_after = {}
    for i in range(np.shape(temp_after)[0]):
        item_after = int((temp_after[i,0] - end_unix_time) // 3600)
        data_hashtag_after[key][item_after] += np.array([1, int(temp_after[i, 1]), int(temp_after[i, 2]), 0, 0])
#         data_hashtag_after[key][item_after][3] = np.max(temp_after[:,2])
        dt_obj_pst = datetime.fromtimestamp(temp_after[i,0], pst_tz)
        data_hashtag_after[key][item_after][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_after not in num_followers_after.keys():
            num_followers_after[item_after] = []
        num_followers_after[item_after].append(temp_after[i,2])
    for i in num_followers_after.keys():
        data_hashtag_after[key][i][3] = np.max(num_followers_after[i])
        
print('done')

gohawks
gopatriots
nfl
patriots
sb49
superbowl
done


In [86]:
# Aggregate data

# Initialize aggregated data variables
data_aggregate_before = np.zeros([num_windows_before, 5])
data_aggregate_during = np.zeros([num_windows_during, 5])
data_aggregate_after = np.zeros([num_windows_after, 5])

# Sum the # of tweets, total # of retweets, and # of followers
for key in hashtags:
    data_aggregate_before[:,0:3] += data_hashtag_before[key][:,0:3]
    data_aggregate_during[:,0:3] += data_hashtag_during[key][:,0:3]
    data_aggregate_after[:,0:3] += data_hashtag_after[key][:,0:3]
# Find the max # of followers for each
data_aggregate_before[:,3] = np.amax([data_hashtag_before[key][:,3] for key in hashtags], axis=0)
data_aggregate_during[:,3] = np.amax([data_hashtag_during[key][:,3] for key in hashtags], axis=0)
data_aggregate_after[:,3] = np.amax([data_hashtag_after[key][:,3] for key in hashtags], axis=0)

# Copy over the same time frames
data_aggregate_before[:,4] = data_hashtag_before['gohawks'][:,4]
data_aggregate_during[:,4] = data_hashtag_during['gohawks'][:,4]
data_aggregate_after[:,4] = data_hashtag_after['gohawks'][:,4]

In [84]:
# print(data_hashtag_after['gohawks'])

### Neural Network

In [87]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [88]:
layer_sizes = [(50, 50), (100, 100), (300, 300), (300, 200, 100), (100, 100, 100, 100)]

In [89]:
y = data_aggregate_before[:,0]
X = np.delete(data_aggregate_before, 0, 1)
mses_before = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    mses_per_layer = []
    kf = KFold(10)
    for trainset, testset in kf.split(X):
        X_train, y_train = X[trainset], y[trainset]
        X_test, y_test = X[testset], y[testset]
        nn.fit(X_train, y_train)
        predicted = nn.predict(X_test)
        mses_per_layer.append(mean_squared_error(y_test, predicted))
    #     print(mean_squared_error(y_test, predicted))
    avg_mse = np.mean(mses_per_layer)
    print('Layer size {} MSE:\n {}'.format(size, np.around(avg_mse)))
    mses_before.append(avg_mse)


Layer size (50, 50) MSE:
 65616082.0
Layer size (100, 100) MSE:
 16585664.0
Layer size (300, 300) MSE:
 14737292.0
Layer size (300, 200, 100) MSE:
 79624094.0
Layer size (100, 100, 100, 100) MSE:
 26057216.0


In [90]:
y = data_aggregate_during[:,0]
X = np.delete(data_aggregate_during, 0, 1)
mses_during = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    mses_per_layer = []
    kf = KFold(10)
    for trainset, testset in kf.split(X):
        X_train, y_train = X[trainset], y[trainset]
        X_test, y_test = X[testset], y[testset]
        nn.fit(X_train, y_train)
        predicted = nn.predict(X_test)
        mses_per_layer.append(mean_squared_error(y_test, predicted))
    #     print(mean_squared_error(y_test, predicted))
    avg_mse = np.mean(mses_per_layer)
    print('Layer size {} MSE:\n {}'.format(size, np.around(avg_mse)))
    mses_during.append(avg_mse)
# print(mses_during)

Layer size (50, 50) MSE:
 4039745540574.0
Layer size (100, 100) MSE:
 834175835959.0
Layer size (300, 300) MSE:
 2496569916038.0
Layer size (300, 200, 100) MSE:
 321499463069.0
Layer size (100, 100, 100, 100) MSE:
 83989000846.0


In [91]:
y = data_aggregate_after[:,0]
X = np.delete(data_aggregate_after, 0, 1)
mses_after = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    mses_per_layer = []
    kf = KFold(10)
    for trainset, testset in kf.split(X):
        X_train, y_train = X[trainset], y[trainset]
        X_test, y_test = X[testset], y[testset]
        nn.fit(X_train, y_train)
        predicted = nn.predict(X_test)
        mses_per_layer.append(mean_squared_error(y_test, predicted))
    #     print(mean_squared_error(y_test, predicted))
    avg_mse = np.mean(mses_per_layer)
    print('Layer size {} MSE:\n {}'.format(size, np.around(avg_mse)))
    mses_after.append(avg_mse)
# print(mses_after)

Layer size (50, 50) MSE:
 1827898035698.0
Layer size (100, 100) MSE:
 188317631495.0
Layer size (300, 300) MSE:
 113673066677.0
Layer size (300, 200, 100) MSE:
 29872622912.0
Layer size (100, 100, 100, 100) MSE:
 13797488099.0
